In [ ]:
example_document = '''Baidu's Apollo Project is one of the world's leading autonomous driving and AI programs, with one of the largest partner ecosystems and over 100 global partners as of 2018, including BYD, Dongfeng, Microsoft, Intel, Nvidia, Daimler AG, ZTE, Grab, Ford, Hyundai and Honda.'''

# GATE

In [ ]:
import requests
url = "https://cloud-api.gate.ac.uk/process-document/annie-named-entity-recognizer"
headers = {'Content-Type': 'text/plain'}
response = requests.post(url, data=example_document, headers=headers).json()

import json
print(json.dumps(response, indent=2))

{
  "text": "Baidu's Apollo Project is one of the world's leading autonomous driving and AI programs, with one of the largest partner ecosystems and over 100 global partners as of 2018, including BYD, Dongfeng, Microsoft, Intel, Nvidia, Daimler AG, ZTE, Grab, Ford, Hyundai and Honda.",
  "entities": {
    "Date": [
      {
        "indices": [
          167,
          171
        ],
        "kind": "date",
        "rule": "TempYear2",
        "ruleFinal": "YearOnlyFinal"
      }
    ],
    "Location": [
      {
        "indices": [
          247,
          251
        ],
        "locType": "city",
        "rule": "Location1",
        "ruleFinal": "LocFinal"
      }
    ],
    "Organization": [
      {
        "indices": [
          8,
          22
        ],
        "orgType": "unknown",
        "rule": "OrgXBase",
        "ruleFinal": "OrgFinal"
      },
      {
        "indices": [
          198,
          207
        ],
        "orgType": "company",
        "rule": "GazOrganization"

In [ ]:
def gate_ner(sentence):
  import requests
  return [(sentence[entity["indices"][0]:entity["indices"][1]] + f" ({entity['gender']})",entity_type) if entity_type == "Person" and "gender" in entity else (sentence[entity["indices"][0]:entity["indices"][1]],entity_type)  for entity_type,entities in requests.post("https://cloud-api.gate.ac.uk/process-document/annie-named-entity-recognizer", data=sentence, headers={'Content-Type': 'text/plain'}).json()["entities"].items() for entity in entities]

In [ ]:
gate_ner(example_document)

[('2018', 'Date'),
 ('Ford', 'Location'),
 ('Apollo Project', 'Organization'),
 ('Microsoft', 'Organization'),
 ('Intel', 'Organization'),
 ('Nvidia', 'Organization'),
 ('Daimler AG', 'Organization'),
 ('Hyundai', 'Organization'),
 ('Honda', 'Organization')]

# NLTK

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
def nltk_ner(document):
  return {(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

In [ ]:
nltk_ner(example_document)

{('AI', 'ORGANIZATION'),
 ('Apollo Project', 'PERSON'),
 ('BYD', 'ORGANIZATION'),
 ('Baidu', 'GPE'),
 ('Daimler AG', 'PERSON'),
 ('Dongfeng', 'PERSON'),
 ('Ford', 'ORGANIZATION'),
 ('Grab', 'PERSON'),
 ('Honda', 'GPE'),
 ('Hyundai', 'PERSON'),
 ('Intel', 'ORGANIZATION'),
 ('Microsoft', 'PERSON'),
 ('Nvidia', 'GPE'),
 ('ZTE', 'ORGANIZATION')}

# Spacy

In [ ]:
!python3 -m spacy download en_core_web_lg
import spacy
sp_lg = spacy.load('en_core_web_lg') 

In [ ]:
def spacy_large_ner(document):
  return {(ent.text.strip(), ent.label_) for ent in sp_lg(document).ents}

In [ ]:
spacy_large_ner(example_document)

{('2018', 'DATE'),
 ('Apollo Project', 'ORG'),
 ('BYD', 'ORG'),
 ('Baidu', 'ORG'),
 ('Daimler AG', 'ORG'),
 ('Dongfeng', 'PERSON'),
 ('Ford', 'ORG'),
 ('Honda', 'ORG'),
 ('Hyundai', 'ORG'),
 ('Intel', 'ORG'),
 ('Microsoft', 'ORG'),
 ('Nvidia', 'ORG'),
 ('ZTE', 'ORG'),
 ('over 100', 'CARDINAL')}

# Flair

In [ ]:
!pip3 install flair

     |████████████████████████████████| 81kB 24.1MB/s 
     |████████████████████████████████| 798kB 42.1MB/s 
     |████████████████████████████████| 133kB 29.2MB/s 
     |████████████████████████████████| 1.0MB 44.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/70/55/6b/c12cf58209b8346f653a04f37dd8f607ab0e85a26238a23420
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
Successfully built flair segtok mpld3 sqlitedict


In [ ]:
from flair.models import SequenceTagger
flair_12class = SequenceTagger.load('ner-ontonotes-fast')
flair_4class = SequenceTagger.load('ner')

2019-05-03 18:22:34,061 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.2/NER-ontoner--h256-l1-b32-%2Bcrawl%2Bnews-forward-fast%2Bnews-backward-fast--v0.2/en-ner-ontonotes-fast-v0.3.pt not found in cache, downloading to /tmp/tmpy8wumt_d


100%|██████████| 1331337776/1331337776 [02:11<00:00, 10094955.31B/s]

2019-05-03 18:24:47,221 copying /tmp/tmpy8wumt_d to cache at /root/.flair/models/en-ner-ontonotes-fast-v0.3.pt


2019-05-03 18:24:54,910 removing temp file /tmp/tmpy8wumt_d
2019-05-03 18:24:54,912 loading file /root/.flair/models/en-ner-ontonotes-fast-v0.3.pt
2019-05-03 18:25:09,807 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/NER-conll03-english/en-ner-conll03-v0.4.pt not found in cache, downloading to /tmp/tmpnejixshj


100%|██████████| 432197603/432197603 [00:47<00:00, 9149328.71B/s]

2019-05-03 18:25:58,338 copying /tmp/tmpnejixshj to cache at /root/.flair/models/en-ner-conll03-v0.4.pt


2019-05-03 18:26:00,534 removing temp file /tmp/tmpnejixshj
2019-05-03 18:26:00,536 loading file /root/.flair/models/en-ner-conll03-v0.4.pt


In [ ]:
def flair_ner(document, model):
  from flair.data import Sentence
  s = Sentence(document)
  model.predict(s)
  entities = s.to_dict(tag_type='ner')
  return [(entity["text"], entity["type"]) for entity in entities["entities"]]

In [ ]:
flair_ner(example_document, flair_4class)

[("Baidu's Apollo Project", 'ORG'),
 ('AI', 'ORG'),
 ('BYD, Dongfeng, Microsoft, Intel, Nvidia, Daimler AG, ZTE, Grab, Ford, Hyundai',
  'ORG'),
 ('Honda.', 'ORG')]

In [ ]:
flair_ner(example_document, flair_12class)

[("Baidu's Apollo Project", 'ORG'),
 ('AI', 'ORG'),
 ('over 100', 'CARDINAL'),
 ('2018,', 'DATE'),
 ('BYD, Dongfeng, Microsoft, Intel, Nvidia, Daimler', 'ORG'),
 ('Hyundai', 'ORG')]

# Deep Pavlov

In [ ]:
!pip3 install deeppavlov
!python3 -m deeppavlov install ner_ontonotes
from deeppavlov import configs, build_model
deeppavlov_ner = build_model(configs.ner.ner_ontonotes, download=True)

     |████████████████████████████████| 604kB 22.4MB/s 
     |████████████████████████████████| 12.2MB 33.0MB/s 
     |████████████████████████████████| 8.0MB 42.2MB/s 
     |████████████████████████████████| 11.8MB 43.1MB/s 
     |████████████████████████████████| 92kB 33.1MB/s 
     |████████████████████████████████| 31.2MB 47.0MB/s 
     |████████████████████████████████| 61kB 30.3MB/s 
     |████████████████████████████████| 12.4MB 48.4MB/s 
     |████████████████████████████████| 51kB 27.0MB/s 
     |████████████████████████████████| 3.4MB 51.8MB/s 
     |████████████████████████████████| 61kB 29.5MB/s 
     |████████████████████████████████| 307kB 51.4MB/s 
     |████████████████████████████████| 4.1MB 55.2MB/s 
     |████████████████████████████████| 51kB 27.3MB/s 
     |████████████████████████████████| 61kB 32.3MB/s 
     |████████████████████████████████| 143kB 59.1MB/s 
     |████████████████████████████████| 2.3MB 45.4MB/s 
     |████████████████████████████████| 7.1MB 31.3

2019-05-03 18:45:42.8 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'ner_ontonotes' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_ontonotes.json'
     |████████████████████████████████| 17.2MB 32.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/3a/1f/86/63c886325bdffa379a7c91499bc9ea6317a4e4e0fc6e2ff1ce
Successfully built gensim
ERROR: flair 0.4.1 has requirement gensim>=3.4.0, but you'll have gensim 2.3.0 which is incompatible.
ERROR: flair 0.4.1 has requirement tqdm>=4.26.0, but you'll have tqdm 4.23.4 which is incompatible.
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 58.4MB 44.0MB/s 
     |████████████████████████████████| 573kB 43.5MB/s 
     |████████████████████████████████| 3.3MB 50.1MB/s 
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.10.0 which is incompatible.
ERROR: goo

2019-05-03 18:46:25.848 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt to /root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt


2019-05-03 18:46:25,848 Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt to /root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt


347MB [00:56, 6.14MB/s]
2019-05-03 18:47:23.683 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_v3_cpu_compatible.tar.gz to /root/.deeppavlov/ner_ontonotes_v3_cpu_compatible.tar.gz


...

2019-05-03 18:48:07,348 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2019-05-03 18:48:14,219 From /usr/local/lib/python3.6/dist-packages/deeppavlov/core/layers/tf_layers.py:861: calling reverse_sequence (from tensorflow.python.ops.array_ops) with seq_dim is deprecated and will be removed in a future version.
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
2019-05-03 18:48:14,238 From /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/deprecation.py:454: calling reverse_sequence (from tensorflow.python.ops.array_ops) with batch_dim is deprecated and will be removed in a future version.
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


2019-05-03 18:48:14.240 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


2019-05-03 18:48:14,240 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-05-03 18:48:15.920 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 50: [loading model from /root/.deeppavlov/models/ner_ontonotes/model]


2019-05-03 18:48:15,920 [loading model from /root/.deeppavlov/models/ner_ontonotes/model]


In [ ]:
def convert_entities(entities):
  ents = set()
  for entity,next_entity in zip(entities,entities[1:] + [(".","O")]):
    word,tag = entity
    if tag != "O":
      ent_position, ent_type = tag.split("-")
      if ent_position == "U":
        ents.add((word,ent_type))
      else:
        if ent_position == "B":
          w = word
        elif ent_position == "I":
          w += " " + word
          if next_entity[1].split("-")[0] != "I":
            ents.add((w,ent_type))
  return ents

def dp_ner(sentence):
  tokens,tags = deeppavlov_ner([sentence])
  return convert_entities([(tok,tg) for token,tag in zip(tokens,tags) for tok,tg in list(zip(token,tag)) ])

In [ ]:
dp_ner(example_document)  

{('Apollo Project', 'ORG'), ('Daimler AG', 'ORG'), ('over 100', 'CARDINAL')}

# Stanford Core NLP

In [ ]:
!pip3 install nltk==3.2.4

     |████████████████████████████████| 1.2MB 22.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/36/f1/5c/f667347d86a3a534ba4c0127eed4389f929916e3ec88bb461a
Successfully built nltk
ERROR: deeppavlov 0.2.0 has requirement nltk==3.2.5, but you'll have nltk 3.2.4 which is incompatible.
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
!wget http://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
!unzip stanford-ner-2015-04-20.zip 

--2019-05-03 18:58:03--  http://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip [following]
--2019-05-03 18:58:03--  https://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176961718 (169M) [application/zip]
Saving to: ‘stanford-ner-2015-04-20.zip.1’

p.1                  52%[=========>          ]  88.66M  3.32MB/s    eta 26s    ^C
Archive:  stanford-ner-2015-04-20.zip
replace stanford-ner-2015-04-20/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from nltk.tag.stanford import StanfordNERTagger
jar = "stanford-ner-2015-04-20/stanford-ner-3.5.2.jar"
model = "stanford-ner-2015-04-20/classifiers/" 
st_3class = StanfordNERTagger(model + "english.all.3class.distsim.crf.ser.gz", jar, encoding='utf8') 
st_4class = StanfordNERTagger(model + "english.conll.4class.distsim.crf.ser.gz", jar, encoding='utf8') 
st_7class = StanfordNERTagger(model + "english.muc.7class.distsim.crf.ser.gz", jar, encoding='utf8') 

/usr/local/lib/python3.6/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  '-tokenizerFactory',
/usr/local/lib/python3.6/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  '-tokenizerFactory',
/usr/local/lib/python3.6/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  '-tokenizerFactory',


In [ ]:
def stanford_ner(document,model):
  if model == 1:
    return [(entity,tag) for entity,tag in st_3class.tag(document.split()) if tag != "O"]
  elif model == 2:
    return [(entity,tag) for entity,tag in st_4class.tag(document.split()) if tag != "O"]
  elif model == 3:
    return [(entity,tag) for entity,tag in st_7class.tag(document.split()) if tag != "O"]

In [ ]:
stanford_ner(example_document,model=1)

[('Hyundai', 'ORGANIZATION')]

In [ ]:
stanford_ner(example_document,model=2)

[('Apollo', 'ORGANIZATION'),
 ('Project', 'ORGANIZATION'),
 ('Daimler', 'ORGANIZATION'),
 ('Hyundai', 'ORGANIZATION')]

In [ ]:
stanford_ner(example_document,model=3)

[('Apollo', 'ORGANIZATION'),
 ('Project', 'ORGANIZATION'),
 ('Daimler', 'ORGANIZATION'),
 ('Hyundai', 'ORGANIZATION'),
 ('Honda.', 'LOCATION')]

# Allen NLP

In [ ]:
!pip3 install allennlp
from allennlp.predictors import Predictor
al = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/fine-grained-ner-model-elmo-2018.12.21.tar.gz")

     |████████████████████████████████| 5.6MB 25.3MB/s 
     |████████████████████████████████| 133kB 26.8MB/s 
     |████████████████████████████████| 245kB 34.2MB/s 
     |████████████████████████████████| 51kB 19.9MB/s 
     |████████████████████████████████| 583kB 43.8MB/s 
     |████████████████████████████████| 51kB 28.8MB/s 
     |████████████████████████████████| 245kB 48.3MB/s 
     |████████████████████████████████| 1.5MB 42.5MB/s 
     |████████████████████████████████| 143kB 44.6MB/s 
     |████████████████████████████████| 92kB 35.2MB/s 
     |████████████████████████████████| 2.1MB 50.4MB/s 
     |████████████████████████████████| 51kB 21.3MB/s 
     |████████████████████████████████| 204kB 52.9MB/s 
     |████████████████████████████████| 61kB 25.4MB/s 
     |████████████████████████████████| 102kB 33.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/f0/47/51/a178b15274ed0db775a1ae9c799ce31e511609c3ab75a7dec5
  Stored in directory: /root/.cache/pip/wheels/b7/8d/e7/a0

100%|██████████| 724601837/724601837 [00:52<00:00, 13728259.04B/s]
/usr/local/lib/python3.6/dist-packages/allennlp/data/token_indexers/token_characters_indexer.py:51: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


In [ ]:
def convert_results(allen_results):
  ents = set()
  for word, tag in zip(allen_results["words"], allen_results["tags"]):
    if tag != "O":
      ent_position, ent_type = tag.split("-")
      if ent_position == "U":
        ents.add((word,ent_type))
      else:
        if ent_position == "B":
          w = word
        elif ent_position == "I":
          w += " " + word
        elif ent_position == "L":
          w += " " + word
          ents.add((w,ent_type))
  return ents

def allennlp_ner(document):
  return convert_results(al.predict(sentence=document))

In [ ]:
allennlp_ner(example_document)

{('2018', 'DATE'),
 ('Apollo Project', 'ORG'),
 ('BYD', 'ORG'),
 ('Baidu', 'ORG'),
 ('Daimler AG', 'ORG'),
 ('Dongfeng', 'ORG'),
 ('Ford', 'ORG'),
 ('Grab', 'ORG'),
 ('Honda', 'ORG'),
 ('Hyundai', 'ORG'),
 ('Intel', 'ORG'),
 ('Microsoft', 'ORG'),
 ('Nvidia', 'ORG'),
 ('ZTE', 'ORG'),
 ('one', 'CARDINAL'),
 ('over 100', 'CARDINAL')}

# Polyglot

In [ ]:
!pip3 install -U git+https://github.com/aboSamoor/polyglot.git@master
!polyglot download embeddings2.en ner2.en
from polyglot.text import Text

  Cloning https://github.com/aboSamoor/polyglot.git (to revision master) to /tmp/pip-req-build-2wnozxsr
  Running command git clone -q https://github.com/aboSamoor/polyglot.git /tmp/pip-req-build-2wnozxsr
     |████████████████████████████████| 14.3MB 34.5MB/s 
     |████████████████████████████████| 215kB 49.4MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-nr0lv35n/wheels/42/d9/73/345c7ae8554299ff8b31635d64eb8455fd591385fa734cdbef
  Stored in directory: /root/.cache/pip/wheels/f3/f9/c7/4fbf1faa6038faf183f6e3ea61f17a5f7eea5ab9a1dd7753fd
  Stored in directory: /root/.cache/pip/wheels/7b/44/44/ec4c5e25e095f02aa0e63ef2bf0cc8badda5877330ffa5fbe4
  Stored in directory: /root/.cache/pip/wheels/3f/45/7e/ccee9f1fe52787595e92641b5645cdf2cb40096749b39b4422
  Stored in directory: /root/.cache/pip/wheels/43/74/e2/92a807d8662f5a0df908ba20887f8ad02bf1b130d3939b65b7
Successfully built polyglot futures pycld2 PyICU morfessor


[polyglot_data] Downloading package embeddings2.en to
[polyglot_data]     /root/polyglot_data...
[polyglot_data] Downloading package ner2.en to /root/polyglot_data...


In [ ]:
def polyglot_ner(document):
  return {(' '.join(entity),entity.tag.split('-')[-1]) for entity in Text(document).entities}

In [ ]:
polyglot_ner(example_document)

{('Daimler AG', 'ORG'),
 ('Honda', 'ORG'),
 ('Hyundai', 'ORG'),
 ('Intel', 'ORG'),
 ('Microsoft', 'ORG'),
 ('Nvidia', 'ORG')}